In [12]:
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

In [13]:
# 設定隨機種子，確保實驗可重現
seed=42

# 設定資料路徑
file_path = './cases/bert_finetune/reviews.txt'

# 讀取資料
with open(file_path, "r", encoding='utf-8') as file:
    # 將每一行資料以 list 型態回傳
    lines = file.readlines()

    # 整合訓練資料
    sentences = []
    labels = []

    # 逐行讀取資料
    for line in lines:
        # 每一行資料的 tab (\t) 作為分隔符號
        parts = line.strip().split('\t')

        # 確保每一行資料都有兩個部分
        if len(parts) == 2:
            sentences.append(parts[0].strip('"'))
            labels.append(int(parts[1]))
        else:
            print(f'格式錯誤的行號: {line}')

In [14]:
# 切分數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    sentences, 
    labels, 
    test_size=0.2, 
    random_state=seed, 
    stratify=labels
)

In [16]:
# 建立 TF-IDF 物件
# tokenizer 預設是 None，表示使用內建的斷詞方式，這裡我們使用 jieba.lcut 來進行中文斷詞
# token_pattern=None 是為了讓自訂的 tokenizer 生效，否則預設會以英文單字的正則表達式來切分
# ngram_range 可調整 n-gram 範圍，
# (1,1) 表示只使用 unigram，
# (2,2) 表示只使用 bigram, 
# (1,2) 表示同時使用 unigram 和 bigram
vectorizer = TfidfVectorizer(
    tokenizer=jieba.lcut, 
    token_pattern=None, 
    ngram_range=(1, 2)
)

'''
fit(...)：從資料學出向量化規則
- 建詞表（vocabulary：有哪些詞/詞組會成為特徵）
- 計算 IDF（每個詞的 inverse document frequency）
- 決定特徵維度與欄位順序

transform(...)：用已學好的規則把新資料轉成向量
- 只把文本映射到既有詞表的欄位
- 用既有的 IDF 權重計算 TF-IDF
- 遇到訓練時沒看過的詞（OOV）就忽略（或不產生新欄位）

fit_transform(...)：
- 等於 fit() + transform() 一次做完
'''

# 對訓練集進行 TF-IDF 轉換
X_train_ft = vectorizer.fit_transform(X_train)

# 轉換測試集
X_test_t = vectorizer.transform(X_test)

In [ ]:
# 建立貝葉斯分類器物件
# alpha: 平滑參數，預設值為 1.0，愈大表示平滑效果愈強，可避免機率為零的問題，愈小表示較少平滑，較接近原始資料分佈
'''
平滑就是把「沒看過」改成「幾乎沒看過」，例如：
- 沒平滑：沒出現 → 機率 0
- 有平滑：沒出現 → 仍給一個很小的機率
這樣模型在測試時遇到新詞或稀有詞，不會一票否決某個類別。
'''
clf = ComplementNB(alpha=1.0)

# 訓練模型
clf = clf.fit(X_train_ft, y_train)

In [18]:
# 進行預測
y_pred = clf.predict(X_test_t)

In [19]:
# 輸出準確率
accuracy_score(y_test, y_pred)

0.722007722007722